# Tutorial: Saving strategies

This tutorial will teach you how to save your processed variables using different *SavingStrategies*.

We will use the loaded variables from the cdf file to showcase how to save them with different formats. We will add additional variables so that we have more to save.

In [1]:
from datetime import datetime, timezone

from astropy import units as u

import el_paso as ep

extraction_infos = [
    ep.ExtractionInfo(
        result_key="Epoch",
        name_or_column="Epoch_Ele",
        unit=u.tt2000,
    ),
    ep.ExtractionInfo(
        result_key="FEDU",
        name_or_column="FEDU",
        unit=(u.cm**2 * u.s * u.sr * u.keV) ** (-1),
    ),
    ep.ExtractionInfo(
        result_key="xGEO",
        name_or_column="Position_Ele",
        unit=u.km,
    ),
]

start_time = datetime(2017, 7, 30, tzinfo=timezone.utc)
end_time = datetime(2017, 8, 1, 23, 59,59, tzinfo=timezone.utc)

file_name_stem = "rbspa_rel04_ect-hope-pa-l3_YYYYMMDD_.{6}.cdf"

ep.download(start_time, end_time,
             save_path=".",
             download_url="https://spdf.gsfc.nasa.gov/pub/data/rbsp/rbspa/l3/ect/hope/pitchangle/rel04/YYYY/",
             file_name_stem=file_name_stem,
             file_cadence="daily",
             method="request",
             skip_existing=True)

variables = ep.extract_variables_from_files(start_time, end_time, "daily",
                                             data_path=".", file_name_stem=file_name_stem,
                                             extraction_infos=extraction_infos)
variables

File already exists, skipping download: rbspa_rel04_ect-hope-pa-l3_20170730_v7.3.0.cdf
File already exists, skipping download: rbspa_rel04_ect-hope-pa-l3_20170731_v7.4.0.cdf
File already exists, skipping download: rbspa_rel04_ect-hope-pa-l3_20170801_v7.3.0.cdf
Extracting variables ...
Concatenating data for Epoch_Ele ...
Concatenating data for FEDU ...
Concatenating data for Position_Ele ...
Concatenating data for Epoch_Ele ...
Concatenating data for FEDU ...
Concatenating data for Position_Ele ...
		Finished in 0.812 seconds


{'Epoch': Variable holding (12474,) data points with metadata: VariableMetadata(unit=Unit("tt2000"), original_cadence_seconds=0, source_files=['rbspa_rel04_ect-hope-pa-l3_20170730_v7.3.0.cdf', 'rbspa_rel04_ect-hope-pa-l3_20170731_v7.4.0.cdf', 'rbspa_rel04_ect-hope-pa-l3_20170801_v7.3.0.cdf'], description='', processing_notes='', standard_name=''),
 'FEDU': Variable holding (12474, 11, 72) data points with metadata: VariableMetadata(unit=Unit("1 / (keV s sr cm2)"), original_cadence_seconds=0, source_files=['rbspa_rel04_ect-hope-pa-l3_20170730_v7.3.0.cdf', 'rbspa_rel04_ect-hope-pa-l3_20170731_v7.4.0.cdf', 'rbspa_rel04_ect-hope-pa-l3_20170801_v7.3.0.cdf'], description='', processing_notes='', standard_name=''),
 'xGEO': Variable holding (12474, 3) data points with metadata: VariableMetadata(unit=Unit("km"), original_cadence_seconds=0, source_files=['rbspa_rel04_ect-hope-pa-l3_20170730_v7.3.0.cdf', 'rbspa_rel04_ect-hope-pa-l3_20170731_v7.4.0.cdf', 'rbspa_rel04_ect-hope-pa-l3_20170801_v7.3.

## Single file strategy

First, we want to save the variables using the *SingleFileStrategy*. This is the simplest way to save variables, as everything is simple put into one file. Dependent on the file ending, different formats will be saved. The units of the variables are not changed in this case.

In [2]:
saving_strategy = ep.saving_strategies.SingleFileStrategy("rbsp_hope_example.pickle")
ep.save(variables, saving_strategy=saving_strategy, start_time=start_time, end_time=end_time, time_var=variables["Epoch"])

Saving file rbsp_hope_example.pickle...
		Finished in 1.773 seconds


Let's inspect what got saved. The variables are turned into simple numpy arrays before saving. You can see that also a *metadata* variable has been saved. We will look closer at metadata in a different tutorial.

In [3]:
import pickle

with open("rbsp_hope_example.pickle", "rb") as f:
    loaded_data = pickle.load(f)
    print("Keys: ", loaded_data.keys())
    print("Metadata: ", loaded_data["metadata"])
    print("xGEO[0,:]: ", loaded_data["xGEO"][0,:] * u.Unit(loaded_data["metadata"]["xGEO"]["unit"]))


Keys:  dict_keys(['Epoch', 'FEDU', 'xGEO', 'metadata'])
Metadata:  {'Epoch': {'unit': 'tt2000', 'original_cadence_seconds': 0, 'source_files': [], 'description': '', 'processing_notes': ''}, 'FEDU': {'unit': '1 / (keV s sr cm2)', 'original_cadence_seconds': 0, 'source_files': [], 'description': '', 'processing_notes': ''}, 'xGEO': {'unit': 'km', 'original_cadence_seconds': 0, 'source_files': [], 'description': '', 'processing_notes': ''}}
xGEO[0,:]:  [-20312.193   -27551.27      1005.89374] km


## Data-org strategy

All data at GFZ is stored under a standard, which we call DataOrg (Data Organization). The standard inlcudes monthly files, where variables are split into separate files to reduce loading times. All files are saved as .mat files. By using the *DataOrgStrategy*, the variables are automatically sorted into the corresponding files, based on the key of the variable_dict. Additionally, the variables are converted into the units as described in the standard.

For using it, we have to store the variables in a dictionary with certain keys, as specified by the standard. 

In [4]:
variables_to_save = {
    "time": variables["Epoch"],
    "Flux": variables["FEDU"],
    "xGEO": variables["xGEO"],
}

saving_strategy = ep.saving_strategies.DataOrgStrategy(".", mission="RBSP", satellite="rbspa", instrument="hope", kext="T89")
ep.save(variables_to_save, saving_strategy, start_time, end_time, time_var=variables["Epoch"])


Saving file rbspa_hope_20170701to20170731_flux_ver4.mat...


/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/save.py:35: UserWarning: Could not find target variable alpha_local!
  target_variables = saving_strategy.get_target_variables(output_file, variables_dict, time_var, interval_start, interval_end)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/utils.py:99: UserWarning: Saving attempted, but product is missing some required variables for output alpha_and_energy!
  result = f(*args, **kw)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/save.py:35: UserWarning: Could not find target variable MLT!
  target_variables = saving_strategy.get_target_variables(output_file, variables_dict, time_var, interval_start, interval_end)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/utils.py:99: UserWarning: Saving attempted, but product is missing some required variables for output mlt!
  result = f(*args, **kw)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_pa

Saving file rbspa_hope_20170701to20170731_xGEO_ver4.mat...


/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/save.py:35: UserWarning: Could not find target variable InvMu!
  target_variables = saving_strategy.get_target_variables(output_file, variables_dict, time_var, interval_start, interval_end)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/utils.py:99: UserWarning: Saving attempted, but product is missing some required variables for output invmu_and_invk!
  result = f(*args, **kw)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/save.py:35: UserWarning: Could not find target variable B_eq!
  target_variables = saving_strategy.get_target_variables(output_file, variables_dict, time_var, interval_start, interval_end)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/utils.py:99: UserWarning: Saving attempted, but product is missing some required variables for output bfield!
  result = f(*args, **kw)
/home/bhaas/Repos/EL-PASO/venv/lib/python3.11/site-packages/el_paso/s

Saving file rbspa_hope_20170801to20170831_flux_ver4.mat...
Saving file rbspa_hope_20170801to20170831_xGEO_ver4.mat...
		Finished in 16.411 seconds


Let's inspect again, what has been saved. First of all, we got many warnings, because we did not provide all the variables which should be saved by the standard. Nevertheless, flux-files and xGEO-files got saved. Note how the metadata struct is again saved and that the units of xGEO got automatically transformed from km to R_E. 

In [5]:
from scipy.io import loadmat
from pathlib import Path

saved_files = sorted(Path("RBSP/rbspa/Processed_Mat_Files").glob("*.mat"))
print("Number of saved files:", len(saved_files))
print("Saved files:", saved_files)

xGEO_data = loadmat(saved_files[1], simplify_cells=True)

print("xGEO data keys:", xGEO_data.keys())
print("xGEO data metadata:", xGEO_data["metadata"]["xGEO"].keys())
print("xGEO[0,:]:", xGEO_data["xGEO"][0,:] * u.Unit(xGEO_data["metadata"]["xGEO"]["unit"]))

Number of saved files: 4
Saved files: [PosixPath('RBSP/rbspa/Processed_Mat_Files/rbspa_hope_20170701to20170731_flux_ver4.mat'), PosixPath('RBSP/rbspa/Processed_Mat_Files/rbspa_hope_20170701to20170731_xGEO_ver4.mat'), PosixPath('RBSP/rbspa/Processed_Mat_Files/rbspa_hope_20170801to20170831_flux_ver4.mat'), PosixPath('RBSP/rbspa/Processed_Mat_Files/rbspa_hope_20170801to20170831_xGEO_ver4.mat')]
xGEO data keys: dict_keys(['__header__', '__version__', '__globals__', 'time', 'xGEO', 'metadata'])
xGEO data metadata: dict_keys(['unit', 'original_cadence_seconds', 'source_files', 'description', 'processing_notes'])
xGEO[0,:]: [-3.1846778  -4.3196673   0.15771057] RE
